#### 手写体识别


In [75]:
import tensorflow as tf
import numpy as np
from tensorflow.examples.tutorials.mnist import input_data;
#设置每一轮训练的batch
batch_size = 100
#设置初始学习率
learning_rate = 0.8
#学习率衰减
learning_rate_decay = 0.999
#最大训练步数
max_steps = 30000

#定义存储训练轮数的变量,其设置为不可训练类型，避免这个变量被计算滑动平均值
training_step = tf.Variable(0,trainable=False)

##### 1. 获取数据集合60000张图片,图片大小为28x28转化为一维数据后成784
``` python
train(train:55000+validation:5000) + test:10000
mnist = train + validation + test
```

In [76]:
mnist = input_data.read_data_sets("/Users/aily/Downloads/文件/百度云/AI数据集合/mnist",one_hot=True)
print("Training data and label size:")
print(mnist.train.images.shape, mnist.train.labels.shape)
print("Testing data and label size:")
print(mnist.test.images.shape, mnist.test.labels.shape)
print("Validating data and label size:")
print(mnist.validation.images.shape, mnist.validation.labels.shape)
print("Example training data",mnist.train.images[0])
print("Example label data",mnist.train.labels[0])

Extracting /Users/aily/Downloads/文件/百度云/AI数据集合/mnist/train-images-idx3-ubyte.gz
Extracting /Users/aily/Downloads/文件/百度云/AI数据集合/mnist/train-labels-idx1-ubyte.gz
Extracting /Users/aily/Downloads/文件/百度云/AI数据集合/mnist/t10k-images-idx3-ubyte.gz
Extracting /Users/aily/Downloads/文件/百度云/AI数据集合/mnist/t10k-labels-idx1-ubyte.gz
Training data and label size:
(55000, 784) (55000, 10)
Testing data and label size:
(10000, 784) (10000, 10)
Validating data and label size:
(5000, 784) (5000, 10)
Example training data [0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0

##### 2. 定义隐藏层和输出层的前向传播方式，激活函数是relu

In [77]:
def hidden_layer(input_tensor, weight1, bias1, weight2, bias2, layer_name):
    layer1 = tf.nn.relu(tf.matmul(input_tensor, weight1) + bias1)
    layer2 = tf.nn.relu(tf.matmul(layer1, weight2) + bias2)
    return layer2

placeholder是占位符，相当于一个形参。只有在被调用时才分配内存单元,在调用结束时,就会释放出所分配的内存单元。

In [78]:
x = tf.placeholder(tf.float32, shape=(None, 784),name="x-input")
y_ = tf.placeholder(tf.float32, shape=(None, 10),name="y-output")

##### 3. 定义权重参数和偏置参数

In [79]:
#生成隐藏层参数其中weights包含784*500=392000个参数
weight1 = tf.Variable(tf.truncated_normal([784,500],stddev=0.1))
bias1 = tf.Variable(tf.constant(0.1, shape=[500]))

#生成输出层参数，其中weight2包含500*10=5000个参数
weight2 = tf.Variable(tf.truncated_normal([500,10],stddev=0.1))
bias2 = tf.Variable(tf.constant(0.1, shape=[10]))

#前向传播计算y值
y = hidden_layer(x,weight1, bias1, weight2, bias2, 'y')

##### 4.使用滑动平均方法更新隐藏层与输出层参数
首先明确一点，TensorFlow中的ExponentialMovingAverage()是针对权重weight和偏差bias的，而不是针对训练集的。如果你现在训练集中实现这个效果，需要自己设计代码。 
为什么要对w和b使用滑动平均模型呢？因为在神经网络中， 
更新的参数时候不能太大也不能太小，更新的参数跟你之前的参数有联系，不能发生突变。一旦训练的时候遇到个“疯狂”的参数，有了滑动平均模型，疯狂的参数就会被抑制下来，回到正常的队伍里。这种对于突变参数的抑制作用，用专业术语讲叫鲁棒性，鲁棒性就是对突变的抵抗能力，鲁棒性越好，这个模型对恶性参数的提抗能力就越强。 
在TensorFlow中，ExponentialMovingAverage()可以传入两个参数：衰减率（decay）和数据的迭代次数（step），这里的decay和step分别对应我们的β和num_updates，所以在实现滑动平均模型的时候，步骤如下： 
- 1、定义训练轮数step 
- 2、然后定义滑动平均的类 
- 3、给这个类指定需要用到滑动平均模型的变量（w和b） 
- 4、执行操作，把变量变为指数加权平均值

[相关解释连接](https://blog.csdn.net/m0_38106113/article/details/81542863)

In [80]:
#初始化一个滑动平均类，衰减率为0.99，为了在训练前期更新的更快，这里提供了num_updates,并设定了训练轮数
averages_class = tf.train.ExponentialMovingAverage(0.99, training_step)

#定义一个更新变量滑动平均值的操作，用tf.trainable_variable()获取所有可以训练的变量列表，也就是所有的w和b
#也就是所有没有指定trainable_variables=False的参数
averages_op = averages_class.apply(tf.trainable_variables())

# 反向传播更新参数之后，再更新每一个参数的滑动平均值，计算y值
average_y = hidden_layer(x, averages_class.average(weight1),
                         averages_class.average(bias1),
                         averages_class.average(weight2), 
                         averages_class.average(bias2), 'average_y' )


##### 5. 计算损失(交叉熵)
``` python
softmax_cross_entropy_with_logits
#可用于输入样本属于多类
sparse_softmax_cross_entropy_with_logits
#更适合输入样本只属于一类
```
[解释可见链接](https://blog.csdn.net/ZJRN1027/article/details/80199248 )

In [81]:
#argmax(input, axis, name, dimension),用于计算每一个样例的预测答案，
#其中input参数y是一个batch_size*10(batch_size行，10列)的二维数组，每
#每一行表示一个样例前向传播的结果，axis=1表示选取最大值操作仅在第一个维度
#中进行，即只在每一行选取最大值对应的下标，所以得到的是长度为batch_size
#的一维数组，这个一维数组中的值就表示了每个样例对应的数字识别结果.
#传入的logits为神经网络输出层的输出，shape为[batch_size，num_classes]，
#传入的label为一个一维的vector，长度等于batch_size，每一个值的取值区间必
#须是[0，num_classes)，其实每一个值就是代表了batch中对应样本的类别。
cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=y, labels=tf.argmax(y_, 1))
#计算L2正则损失
regularizer = tf.contrib.layers.l2_regularizer(0.0001)
#计算模型正则化损失
regularization = regularizer(weight1) + regularizer(weight2)
#计算总损失
loss = tf.reduce_mean(cross_entropy) + regularization
#用指数衰减法设置学习率，这里staircase参数采用默认的false，即学习率连续衰减法
learning_rate = tf.train.exponential_decay(learning_rate,
                                           training_step,
                                           mnist.train.num_examples/batch_size, 
                                           learning_rate_decay)
#优化损失
train_step = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=training_step)
#训练模型时既需要反向传播更新神经网络中的参数，也需更新每一个窗口的滑动平均值
#以下两种做法都可以
train_op = tf.group(train_step, averages_op)
# with tf.control_dependencies([train_step, averages_op]):
#     train_step = tf.no_op(name="train")
    

##### 6. 检验数据，得出准确率

In [82]:
#检查使用了滑动平均值模型的神经网络前向传播是否正确
#equal(X,y)对比每一维是否相同
crorent_prediction = tf.equal(tf.argmax(average_y, 1), tf.argmax(y_, 1))
#cast(x, DstT, name)这里用于将布尔型转换为float32，之后对float32求平均，就是正确率
accuracy = tf.reduce_mean(tf.cast(crorent_prediction, tf.float32))

##### 7. 创建会话开始训练

In [83]:
with tf.Session() as sess:
    #初始化变量
    tf.global_variables_initializer().run()
    #准备验证数据
    validate_feed = {x:mnist.validation.images, y_:mnist.validation.labels}
    #准备测试数据
    test_feed = {x:mnist.test.images, y_:mnist.test.labels}
    for i in range(max_steps):
        if i % 1000 == 0:
            #计算滑动平均在验证集结果，输出百分比结果
            validate_accuracy = sess.run(accuracy, feed_dict=validate_feed)
            print("After %d training steps, validation accuracy using average model is %g%%"%(i, validate_accuracy * 100))
            #一轮使用一个batch的训练数据，并进行训练input_data.read_data_sets()函数提供了train.next
            xs,ys = mnist.train.next_batch(batch_size = 100)
            sess.run(train_op, feed_dict={x:xs,y_:ys})
    #检验最终正确率
    test_accuracy = sess.run(accuracy, feed_dict = test_feed)
    print("After %d training steps, test accuracy using average model is %g%%"%(max_steps, test_accuracy * 100))


After 0 training steps, validation accuracy using average model is 9.44%
After 1000 training steps, validation accuracy using average model is 9.44%
After 2000 training steps, validation accuracy using average model is 18.96%
After 3000 training steps, validation accuracy using average model is 18.52%
After 4000 training steps, validation accuracy using average model is 17.78%
After 5000 training steps, validation accuracy using average model is 9.58%
After 6000 training steps, validation accuracy using average model is 8.68%
After 7000 training steps, validation accuracy using average model is 16.14%
After 8000 training steps, validation accuracy using average model is 11.26%
After 9000 training steps, validation accuracy using average model is 9.58%
After 10000 training steps, validation accuracy using average model is 9.58%
After 11000 training steps, validation accuracy using average model is 9.58%
After 12000 training steps, validation accuracy using average model is 9.58%
After 1